In [1]:
import json
from typing import List, Dict, Set

# Load Data
DATA_PATH = "../data/massive_data.json"

def load_data(filepath):
    with open(filepath, "r") as f:
        return json.load(f)

class RecommendationEngine:
    def __init__(self, data):
        self.users = data["users"]
        self.pages = {p["id"]: p["name"] for p in data["pages"]}
        
        # Pre-compute lookups for O(1) access
        self.user_map = {u["id"]: u for u in self.users}
        self.friend_map = {u["id"]: set(u["friends"]) for u in self.users}
        self.likes_map = {u["id"]: set(u["liked_pages"]) for u in self.users}

    def recommend_friends(self, user_id: int) -> List[str]:
        """
        Logic: Friend of a Friend (Triadic Closure).
        If A knows B, and B knows C, suggest C to A.
        """
        if user_id not in self.friend_map:
            return []
            
        direct_friends = self.friend_map[user_id]
        candidates = {} # {user_id: mutual_count}

        for friend_id in direct_friends:
            # Check friends of friends
            friends_of_friend = self.friend_map.get(friend_id, set())
            for candidate_id in friends_of_friend:
                if candidate_id != user_id and candidate_id not in direct_friends:
                    candidates[candidate_id] = candidates.get(candidate_id, 0) + 1
        
        # Sort by mutual friends count (High relevance first)
        sorted_recommendations = sorted(candidates.items(), key=lambda x: x[1], reverse=True)
        
        # Return names instead of just IDs
        return [self.user_map[uid]["name"] for uid, _ in sorted_recommendations[:5]]

    def recommend_pages(self, user_id: int) -> List[str]:
        """
        Logic: Collaborative Filtering (User-Based).
        Suggest pages liked by friends that the user hasn't liked yet.
        """
        if user_id not in self.user_map:
            return []
            
        my_likes = self.likes_map[user_id]
        friends = self.friend_map[user_id]
        page_scores = {}

        for friend_id in friends:
            friend_likes = self.likes_map.get(friend_id, set())
            for page_id in friend_likes:
                if page_id not in my_likes:
                    # Score increases if more friends like the same page
                    page_scores[page_id] = page_scores.get(page_id, 0) + 1
        
        sorted_pages = sorted(page_scores.items(), key=lambda x: x[1], reverse=True)
        return [self.pages[pid] for pid, _ in sorted_pages[:5]]

# --- DRIVER CODE ---
if __name__ == "__main__":
    data = load_data(DATA_PATH)
    engine = RecommendationEngine(data)
    
    # Test for User ID 1 (Amit)
    TARGET_USER = 1
    user_name = engine.user_map[TARGET_USER]["name"]
    
    print(f"Analyzing Network for: {user_name} (ID: {TARGET_USER})\n")
    
    rec_friends = engine.recommend_friends(TARGET_USER)
    print(f"People You May Know: {rec_friends}")
    
    rec_pages = engine.recommend_pages(TARGET_USER)
    print(f"Pages You Might Like: {rec_pages}")

Analyzing Network for: Amit (ID: 1)

People You May Know: ['Kunal', 'Anjali', 'Ravi', 'Sneha', 'Arjun']
Pages You Might Like: ['AI & ML Community', 'Web Dev Hub', 'Blockchain Innovators', 'Cybersecurity Experts']
